In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.linalg
import scipy

print(np.__version__)
print(matplotlib.__version__)
print(scipy.__version__)

1.26.0
3.8.0
1.11.2


In [2]:
def normVector(v):
    return np.linalg.norm(v)

def __calcResidual(A, f, X):
    r = np.matrix(np.zeros((n + 1, 1)))
    r = A * X - f
    
    return r

def calcResidual(A, f, X):
    return normVector(__calcResidual(A, f, X))

In [3]:
def makePlot(y, title):
    plt.figure(figsize=(20, 10))

    x = np.arange(len(y))
    plt.plot(x, y, 'o--')
    plt.xlabel(r'$ iteration $', fontsize=16)
    plt.ylabel(r'$ ||r|| $', fontsize=16)

    plt.title(title, fontsize=16)
    plt.grid()
    #plt.legend(fontsize=16)
    plt.show()

## Non-linear equation

In [4]:
def equation_sim(phi, x0, accuracy):
    rs = list()
    x = x0

    while True:
        prev_x = x
        x = phi(x)

        rs.append(abs(prev_x - x))
        if rs[-1] < accuracy:
            break

    return x, rs

def equation_newton(u, x0, accuracy):
    pass

In [7]:
x, rs = equation_sim(lambda x: 1/5 * (-2*(x**2) + 3), 0, 1e-5)
x

0.5000017851488853